In [ ]:
# 環境設置
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException
# ---------- 等待頁面加載完成
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# ----------
import pandas as pd
import pandas.io.formats.excel
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
from urllib.request import HTTPError
from bs4 import BeautifulSoup
import time

In [ ]:
# 國立宜蘭大學、佛光大學
def webpac_jsp_crawler(url, ISBN, driver, wait):
    try:
        # 前往《館藏系統》頁面
        driver.get(url)

        # 等待＜下拉式選單＞元素出現
        wait.until(EC.presence_of_element_located((By.NAME, "search_field")))

        # 定位＜下拉式選單＞，以 ISBN 搜尋方式搜尋
        select = Select(driver.find_element_by_name("search_field"))
        select.select_by_visible_text(u"ISBN")

        # 定位＜搜尋＞，輸入 IBSN 後，按下 ENTER 鍵
        search_input = driver.find_element_by_name("search_input")
        search_input.send_keys(ISBN)
        search_input.submit()

        # 等待＜表格＞元素出現
        wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "table.order")))

        # 取得 tgt
        tgt = pd.read_html(driver.page_source, encoding="utf-8")[-2]

        return tgt
    except:
        print(f"「{url}」無法爬取！")

In [ ]:
tgt

In [ ]:
def click_more_btn(driver):
    while True:
        try:
            time.sleep(1)
            driver.find_element_by_css_selector("a.btnstyle.bluebg3.morewidth").click()
        except NoSuchElementException:
            break
    tgt = pd.read_html(driver.page_source, encoding="utf-8")[0]
    return tgt

In [ ]:
# 宜蘭縣公共圖書館、桃園市立圖書館
def webpac_gov_crawler(url, ISBN, driver):
    tgt = []
    
    # 進入《館藏系統》頁面
    driver.get(url)

    # 點選進階查詢
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="center"]/div/section[1]/div/div/div/form/div[3]/div[1]/a').click()

    # 選擇以 ISBN 查詢，並輸入欲查書的 ISBN，按下 ENTER
    time.sleep(1)
    select = Select(driver.find_element_by_name("searchField"))
    select.select_by_visible_text(u"ISBN")
    search_input = driver.find_element_by_name("searchInput")
    search_input.send_keys(ISBN)
    search_input.submit()

    # 計算搜尋結果的數量
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    results = len(soup.find_all("div", "booklist"))

    # 依照搜尋結果的數量分成一筆和多筆兩種情形
    if results == 0:
        tgt = click_more_btn(driver)
        return tgt
    else:
        book_list = soup.find_all("div", "booklist")
        for li in book_list:
            href = url + li.find("h2").find("a")["href"]
            driver.get(href)
            time.sleep(2)
            tgt.append(click_more_btn(driver))
            tgt = pd.concat(tgt, axis=0, ignore_index=True)
            return tgt